In [ ]:
from data import LoTTSDataset
from torchvision.transforms import v2
import torch

transforms = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32),
        v2.Resize(size=(64, 64)),
    ]
)

data = LoTTSDataset(
    data_folder="./data/lotssdr2/public",  # Change this to where you saved your data
    cutout_scaling=1.5,
    transform=transforms,
)

for i in range(len(data)):
    if i > 10:
        break
    data.plot(
        i,
        contours=True,
        sigma_name="Isl_rms",
        min_sigma=2,
        title=data.df.iloc[i]["Source_Name"] + data.df.iloc[i]["S_Code"],
    )

data.df.head()